# Reconstruct the circuit for testing
### Reconstruct previously generated quantum circuits that were stored in a csv file

In [1]:
# Import required libraries
from qiskit import *
import pandas as pd
from qiskit import transpile
import random
import csv
import ast
import numpy as np
from qiskit.providers.aer import AerSimulator
import time
import itertools
import pickle
import progressbar
import pathlib
from IPython.display import clear_output
from qiskit.tools.monitor import job_monitor

# Load IBM Quantum Experience account
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
provider = IBMQ.load_account()
provider = IBMQ.get_provider(hub='**********', group='*********', project='main')

# Select backend
backend = provider.get_backend('ibm_perth')

# Set the number of qubits
qubitCount = 7

# Create noise model from backend
noise_model = NoiseModel.from_backend(backend)

# Get coupling map from backend
coupling_map = backend.configuration().coupling_map

# Get basis gates from noise model
basis_gates = noise_model.basis_gates

# Define available gates and their costs
Gates = ['rz', 'rx', 'ry', 'sx', 'x', 'y', 'z', 'h', 'cx', 'swap']
gateCosts = [1, 1, 1, 1, 1, 1, 1, 2, 5, 11]

# Define field names for CSV files
fields1 = ['Cost', 'Gate Count', 'Optimization Level', 'Runtime', 'Transpile Time']
fields2 = ['Circuit Qubit', 'Circuit Gate', 'ActCost', 'Act Gate Count', 'sim Cost 0', 'sim Gate Count 0', 'sim Runtime 0', 'sim Transpile Time 0',
           'sim Cost 1', 'sim Gate Count 1', 'sim Runtime 1', 'sim Transpile Time 1',
           'sim Cost 2', 'sim Gate Count 2', 'sim Runtime 2', 'sim Transpile Time 2',
           'sim Cost 3', 'sim Gate Count 3', 'sim Runtime 3', 'sim Transpile Time 3',
           'Cost 0', 'Gate Count 0', 'Runtime 0', 'Transpile Time 0',
           'Cost 1', 'Gate Count 1', 'Runtime 1', 'Transpile Time 1',
           'Cost 2', 'Gate Count 2', 'Runtime 2', 'Transpile Time 2',
           'Cost 3', 'Gate Count 3', 'Runtime 3', 'Transpile Time 3']

# Load MLModel
filename = 'MLModels//PerthSimRegModel.sav'
reg = pickle.load(open(filename, 'rb'))


### Functions

In [2]:
def QCtoDF(qc):
    # Convert quantum circuit to DataFrame
    string = qc.qasm()
    circuit = string.split(';')
    circuit = circuit[3:]
    circuit.pop(len(circuit) - 1)
    with open(r"gatesTemp.csv", 'w', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(['Gate', 'Qubit'])
        for i in range(len(circuit)):
            circuit[i] = circuit[i].replace("\n", '')
            temp = circuit[i].split(' ')
            for j in range(len(temp[1])):
                temp[1] = temp[1].replace("q[", '')
                temp[1] = temp[1].replace("]", '')
            if temp[0] != 'measure' and temp[0] != 'barrier' and temp[0] != 'creg':
                writer.writerow(temp)
    df = pd.DataFrame(pd.read_csv(r"gatesTemp.csv"))
    return df

def unique(list1):
    # Return unique elements from the list
    unique_list = []
    for x in list1:
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

def depthFinder(df):
    # Find the depth of the circuit
    qubits = df['Qubit'].value_counts()
    qubits = dict(qubits)
    lst = list(qubits.items())
    if len(lst) > 1:
        qnum = []
        for j in range(len(lst)):
            strList = str(lst[j][0]).split(',')
            qnum.append(strList[0])
        qnum = unique(qnum)
        counts = []
        for num in qnum:
            for j in range(len(lst)):
                strList = str(lst[j][0]).split(',')
                if strList[0] == num:
                    for times in range(lst[j][1]):
                        counts.append(strList[0])
        return len(counts)
    elif len(lst) > 0:
        return lst[0][1]
    else:
        return 0

def ComplexityFinder(qc):
    # Calculate the complexity of the quantum circuit
    df = QCtoDF(qc)
    depth = depthFinder(df)
    cost = 0
    for i in df['Gate']:
        if i in Gates:
            cost += int(gateCosts[Gates.index(i)])
        else:
            cost += int(gateCosts[Gates.index(i[:2])])
    return cost, depth

def binary_search(cost, name):
    # Perform binary search
    low = 0
    high = int((np.math.factorial(4 + name - 1)) / (np.math.factorial(name) * (np.math.factorial(3)))) - 1
    mid = 0
    while low <= high:
        mid = (high + low) // 2
        try:
            dfFull = pd.read_csv("D:/Documents/Combinations/combo" + str(name) + ".csv", skiprows=mid, nrows=1)
        except:
            with open(r"ErrorLog.csv", 'w', newline='', encoding='UTF8') as f:
                writer = csv.writer(f)
                writer.writerow([name])
                return -1
        if dfFull.loc[0][0] < cost:
            low = mid + 1
        elif dfFull.loc[0][0] > cost:
            high = mid - 1
        else:
            return mid
    return -1

def Average(lst):
    # Calculate the average of a list
    return sum(lst) / len(lst)


In [3]:
def check(x):
    return x and [x[0]]*len(x) == x

In [18]:
# Selector Function
def selector(qc):
    # Start time
    start = time.time()
    runs = 1

    # Optimization level 3
    TT = []
    Cost = []
    GC = []
    RT = []
    for run in range(runs):
        start_time = time.time()
        tqc3 = transpile(qc, backend, optimization_level=3)
        TT3 = time.time() - start_time
        TT.append(TT3)
        Cost3, GC3 = ComplexityFinder(tqc3)
        Cost.append(Cost3)
        GC.append(GC3)
        RT3 = reg.predict([[Cost3, GC3, 3, TT3]])
        RT.append(RT3)
    TT3 = Average(TT)
    Cost3 = Average(Cost)
    GC3 = Average(GC)
    RT3 = Average(RT)
    s3 = [Cost3, GC3, RT3[0], TT3]

    # Optimization level 2
    TT = []
    Cost = []
    GC = []
    RT = []
    for run in range(runs):
        start_time = time.time()
        tqc2 = transpile(qc, backend, optimization_level=2)
        TT2 = time.time() - start_time
        TT.append(TT2)
        Cost2, GC2 = ComplexityFinder(tqc2)
        Cost.append(Cost2)
        GC.append(GC2)
        RT2 = reg.predict([[Cost2, GC2, 2, TT2]])
        RT.append(RT2)
    TT2 = Average(TT)
    Cost2 = Average(Cost)
    GC2 = Average(GC)
    RT2 = Average(RT)
    s2 = [Cost2, GC2, RT2[0], TT2]

    # Optimization level 1
    TT = []
    Cost = []
    GC = []
    RT = []
    for run in range(runs):
        start_time = time.time()
        tqc1 = transpile(qc, backend, optimization_level=1)
        TT1 = time.time() - start_time
        TT.append(TT1)
        Cost1, GC1 = ComplexityFinder(tqc1)
        Cost.append(Cost1)
        GC.append(GC1)
        RT1 = reg.predict([[Cost1, GC1, 1, TT1]])
        RT.append(RT1)
    TT1 = Average(TT)
    Cost1 = Average(Cost)
    GC1 = Average(GC)
    RT1 = Average(RT)
    s1 = [Cost1, GC1, RT1[0], TT1]

    # Optimization level 0
    TT = []
    Cost = []
    GC = []
    RT = []
    for run in range(runs):
        start_time = time.time()
        tqc0 = transpile(qc, backend, optimization_level=0)
        TT0 = time.time() - start_time
        TT.append(TT0)
        Cost0, GC0 = ComplexityFinder(tqc0)
        Cost.append(Cost0)
        GC.append(GC0)
        RT0 = reg.predict([[Cost0, GC0, 0, TT0]])
        RT.append(RT0)
    TT0 = Average(TT)
    Cost0 = Average(Cost)
    GC0 = Average(GC)
    RT0 = Average(RT)
    s0 = [Cost0, GC0, RT0[0], TT0]

    # Get runtime of each optimization level for this qc
    tqc3RT = RT3[0] + TT3
    tqc2RT = RT2[0] + TT2
    tqc1RT = RT1[0] + TT1
    tqc0RT = RT0[0] + TT0
    RTs = [tqc0RT, tqc1RT, tqc2RT, tqc3RT]
    GCs = [GC0, GC1, GC2, GC3]
    Costs = [Cost0, Cost1, Cost2, Cost3]

    # Cascade choice: RT, Gatecount, Cost
    optLvls = [0, 1, 2, 3]
    worst = RTs.index(max(RTs))
    optLvls.pop(worst)
    RTs.pop(worst)
    GCs.pop(worst)
    Costs.pop(worst)

    if check(Costs):
        worst = RTs.index(max(RTs))
    else:
        worst = Costs.index(max(Costs))
    optLvls.pop(worst)
    RTs.pop(worst)
    GCs.pop(worst)
    Costs.pop(worst)

    if check(GCs):
        worst = RTs.index(max(RTs))
    else:
        worst = GCs.index(max(GCs))
    optLvls.pop(worst)
    best = optLvls[0]

    print("old best:", best)

    # Check if any RTs are too close to be distinguished
    RTs = [tqc0RT, tqc1RT, tqc2RT, tqc3RT]
    GCs = [GC0, GC1, GC2, GC3]
    Costs = [Cost0, Cost1, Cost2, Cost3]
    alternatives = []
    for i in range(len(RTs)):
        if i != best:
            if (RTs[i] - RTs[best]) / RTs[best] * 100 <= 30 and GCs[i] < GCs[best] and Costs[i] < Costs[best]:
                best = i
            elif (RTs[i] - RTs[best]) / RTs[best] * 100 <= 5 and GCs[i] <= GCs[best] and Costs[i] <= Costs[best]:
                alternatives.append(i)

    print("RTs:", RTs)
    print("csv GCs:", GCs)
    print("Costs:", Costs)

    # Return the best optimization level, alternatives, and statistics
    return best, alternatives, s3, s2, s1, s0, time.time() - start


### Circuit Generation

In [19]:
complexDF = pd.read_csv("QMachineData/QCDataRealFull1.csv")
# complexDF2 = pd.read_csv("QMachineData//PerthDataRealFull3.csv")
# complexDF3 = pd.read_csv("QMachineData/PLDataRealFull1.csv")
# DF = complexDF.loc[(complexDF['ActCost']== 3) & (complexDF['Act Gate Count'] == 2)]
# DF
# DF.values.tolist()[0]

In [89]:
# Set the initial variables
shots = 1024
runs = 1

simVal = []
simAlts = []
controlVal = []
controlAlts = []

moneySaved = []

maxComboLength = 290

# Read the data from the CSV file
df = pd.read_csv("QMachineData/PerthRand1.csv")

# Iterate over each row in the DataFrame
for row in df.iterrows():
    # Clear the output and display the current cost and total number of gates
    clear_output(wait=True)
    cost = row[1]["ActCost"]
    totalGates = row[1]["Act Gate Count"]
    print("Cost:", cost)
    print("Total Number of Gates:", totalGates)

    # Create a quantum circuit based on the circuit information in the row
    qc = QuantumCircuit(qubitCount)
    qubits = row[1]["Circuit Qubit"]
    gate = row[1]["Circuit Gate"]
    qubits = ast.literal_eval(qubits)
    gate = ast.literal_eval(gate)
    for idx in range(len(gate)):
        if gate[idx] == 'cx' or gate[idx] == 'swap':
            pair = qubits[idx].split(',')
            getattr(qc, gate[idx])(int(pair[0]), int(pair[1]))
        else:
            getattr(qc, gate[idx])(int(qubits[idx]))

    qc.measure_all()
    print(qc)

    # Simulator Optimization
    RTTimes = []
    TTTimes = []
    GCs = []
    costs = []

    # Retrieve the simulation times, gate counts, and costs for each optimization level
    RTTimes.append(row[1]["sim Runtime 0"])
    TTTimes.append(row[1]["sim Transpile Time 0"])
    GCs.append(row[1]["sim Gate Count 0"])
    costs.append(row[1]["sim Cost 0"])

    RTTimes.append(row[1]["sim Runtime 1"])
    TTTimes.append(row[1]["sim Transpile Time 1"])
    GCs.append(row[1]["sim Gate Count 1"])
    costs.append(row[1]["sim Cost 1"])

    RTTimes.append(row[1]["Runtime 2"])
    TTTimes.append(row[1]["Transpile Time 2"])
    GCs.append(row[1]["Gate Count 2"])
    costs.append(row[1]["Cost 2"])

    RTTimes.append(row[1]["sim Runtime 3"])
    TTTimes.append(row[1]["sim Transpile Time 3"])
    GCs.append(row[1]["sim Gate Count 3"])
    costs.append(row[1]["sim Cost 3"])

    # Cascade Method for simulator optimization
    tqc3RT = float(RTTimes[3]) + float(TTTimes[3])
    tqc2RT = float(RTTimes[2]) + float(TTTimes[2])
    tqc1RT = float(RTTimes[1]) + float(TTTimes[1])
    tqc0RT = float(RTTimes[0]) + float(TTTimes[0])
    RTs = [tqc0RT, tqc1RT, tqc2RT, tqc3RT]

    print("Simulated Times: ", RTs)
    print("Simulated GCs: ", GCs)
    print("Simulated Costs: ", costs)

    RTs2 = [RTs[0], RTs[1], RTs[2], RTs[3]]
    GCs2 = [GCs[0], GCs[1], GCs[2], GCs[3]]
    costs2 = [costs[0], costs[1], costs[2], costs[3]]

    optLvls = [0, 1, 2, 3]
    worst = RTs.index(max(RTs))
    optLvls.pop(worst)
    RTs.pop(worst)
    GCs.pop(worst)
    costs.pop(worst)

    if check(costs):
        worst = RTs.index(max(RTs))
    else:
        worst = costs.index(max(costs))
    optLvls.pop(worst)
    RTs.pop(worst)
    GCs.pop(worst)
    costs.pop(worst)

    if check(GCs):
        worst = RTs.index(max(RTs))
    else:
        worst = GCs.index(max(GCs))
    optLvls.pop(worst)
    simbest = optLvls[0]
    print("old best: ", best)

    # Check if any RTs are too close to be distinguished
    alternatives = []
    for i in range(len(RTs2)):
        if i != simbest:
            if (RTs2[i] - RTs2[simbest]) / RTs2[simbest] * 100 <= 30 and GCs2[i] < GCs2[simbest] and costs2[i] < costs2[simbest]:
                simbest = i
            elif (RTs2[i] - RTs2[simbest]) / RTs2[simbest] * 100 <= 5 and GCs2[i] <= GCs2[simbest] and costs2[i] <= costs2[simbest]:
                alternatives.append(i)

    print("Chosen Sim Optimization Level: ", simbest)
    print("Alternative Optimization Lvls: ", alternatives)
    simVal.append(simbest)
    simAlts.append(alternatives)

    # Control Optimization (Real Quantum Hardware)
    RTTimes = []
    TTTimes = []
    GCs = []
    costs = []

    # Retrieve the real quantum hardware times, gate counts, and costs for each optimization level
    RTTimes.append(row[1]["Runtime 0"])
    TTTimes.append(row[1]["Transpile Time 0"])
    GCs.append(row[1]["Gate Count 0"])
    costs.append(row[1]["Cost 0"])

    RTTimes.append(row[1]["Runtime 1"])
    TTTimes.append(row[1]["Transpile Time 1"])
    GCs.append(row[1]["Gate Count 1"])
    costs.append(row[1]["Cost 1"])

    RTTimes.append(row[1]["Runtime 2"])
    TTTimes.append(row[1]["Transpile Time 2"])
    GCs.append(row[1]["Gate Count 2"])
    costs.append(row[1]["Cost 2"])

    RTTimes.append(row[1]["Runtime 3"])
    TTTimes.append(row[1]["Transpile Time 3"])
    GCs.append(row[1]["Gate Count 3"])
    costs.append(row[1]["Cost 3"])

    # Cascade Method for control optimization
    tqc3RT = float(RTTimes[3]) + float(TTTimes[3])
    tqc2RT = float(RTTimes[2]) + float(TTTimes[2])
    tqc1RT = float(RTTimes[1]) + float(TTTimes[1])
    tqc0RT = float(RTTimes[0]) + float(TTTimes[0])
    RTs = [tqc0RT, tqc1RT, tqc2RT, tqc3RT]

    print("Simulated Times: ", RTs)
    print("Simulated GCs: ", GCs)
    print("Simulated Costs: ", costs)

    RTs2 = [RTs[0], RTs[1], RTs[2], RTs[3]]
    GCs2 = [GCs[0], GCs[1], GCs[2], GCs[3]]
    costs2 = [costs[0], costs[1], costs[2], costs[3]]

    optLvls = [0, 1, 2, 3]
    worst = RTs.index(max(RTs))
    optLvls.pop(worst)
    RTs.pop(worst)
    GCs.pop(worst)
    costs.pop(worst)

    if check(costs):
        worst = RTs.index(max(RTs))
    else:
        worst = costs.index(max(costs))
    optLvls.pop(worst)
    RTs.pop(worst)
    GCs.pop(worst)
    costs.pop(worst)

    if check(GCs):
        worst = RTs.index(max(RTs))
    else:
        worst = GCs.index(max(GCs))
    optLvls.pop(worst)
    best = optLvls[0]
    print("old best: ", best)

    # Check if any RTs are too close to be distinguished
    alternatives = []
    for i in range(len(RTs2)):
        if i != best:
            if (RTs2[i] - RTs2[best]) / RTs2[best] * 100 <= 30 and GCs2[i] < GCs2[best] and costs2[i] < costs2[best]:
                best = i
            elif (RTs2[i] - RTs2[best]) / RTs2[best] * 100 <= 5 and GCs2[i] <= GCs2[best] and costs2[i] <= costs2[best]:
                alternatives.append(i)

    print("Chosen Control Optimization Level: ", best)
    print("Alternative Optimization Lvls: ", alternatives)
    controlVal.append(best)
    controlAlts.append(alternatives)
    moneySaved.append(RTs2[simbest] - RTs2[1])
    print("--------------------------------------------------------------")


Cost: 41
Total Number of Gates: 9
        ┌───┐                   ░ ┌─┐                  
   q_0: ┤ X ├──────────■────────░─┤M├──────────────────
        └─┬─┘┌───┐     │  ┌───┐ ░ └╥┘┌─┐               
   q_1: ──┼──┤ X ├─X───┼──┤ X ├─░──╫─┤M├───────────────
          │  ├───┤ │   │  └───┘ ░  ║ └╥┘┌─┐            
   q_2: ──┼──┤ Y ├─┼───┼────X───░──╫──╫─┤M├────────────
          │  └───┘ │ ┌─┴─┐  │   ░  ║  ║ └╥┘┌─┐         
   q_3: ──┼────────X─┤ X ├──┼───░──╫──╫──╫─┤M├─────────
          │  ┌───┐   └───┘  │   ░  ║  ║  ║ └╥┘┌─┐      
   q_4: ──■──┤ X ├──────────X───░──╫──╫──╫──╫─┤M├──────
             └─┬─┘              ░  ║  ║  ║  ║ └╥┘┌─┐   
   q_5: ───────■────────────────░──╫──╫──╫──╫──╫─┤M├───
             ┌───┐              ░  ║  ║  ║  ║  ║ └╥┘┌─┐
   q_6: ─────┤ Y ├──────────────░──╫──╫──╫──╫──╫──╫─┤M├
             └───┘              ░  ║  ║  ║  ║  ║  ║ └╥┘
meas: 7/═══════════════════════════╩══╩══╩══╩══╩══╩══╩═
                                   0  1  2  3  4  5  6 
Simulated Time

In [90]:
sum(moneySaved)

2.244367599487301

In [79]:
len(controlVal)

151

In [80]:
len(simVal)

151

In [81]:
# Get Accuracy
# simVal is the array of selected optimization levels by my Selector function
# controlVal is the array of selected optimization levels by the control
# sel 93.05%
# Sim 84.44%

correct = 0
key = []

# Iterate over the optimization levels selected by the Selector function and the control
for i in range(len(simVal)):
    point = False
    
    # Check if the selected optimization levels match
    if simVal[i] == controlVal[i]:
        correct += 1
        key.append(1)
        point = True
    
    # Check if the selected optimization level by the Selector function matches any alternative optimization level by the control
    for controlAlt in controlAlts[i]:
        if simVal[i] == controlAlt and point == False:
            correct += 1
            key.append(1)
            point = True
            break
    
    # Check if any alternative optimization level by the Selector function matches the selected optimization level by the control
    for alt in simAlts[i]:
        if alt == controlVal[i] and point == False:
            correct += 1
            key.append(1)
            point = True
            break
    
    # Check if any alternative optimization level by the Selector function matches any alternative optimization level by the control
    for alt in simAlts[i]:
        for controlAlt in controlAlts[i]:
            if alt == controlAlt and point == False:
                correct += 0.5
                point = True
                key.append(0.5)
                break
    
    # If no match is found, append 0 to the key list
    if point == False:
        key.append(0)

# Save the key list using pickle
with open("QMachineData/ListData/key", "wb") as fp:
    pickle.dump(key, fp)

# Calculate and print the accuracy
print("Accuracy: ", 100 * (correct / (len(simVal))))


Accuracy:  84.43708609271523


In [ ]:
##Cascade Method
# tqc3RT = float(RTTimes[3]) + float(TTTimes[3])
#     tqc2RT = float(RTTimes[2]) + float(TTTimes[2])
#     tqc1RT = float(RTTimes[1]) + float(TTTimes[1])
#     tqc0RT = float(RTTimes[0]) + float(TTTimes[0])
#     RTs = [tqc0RT, tqc1RT, tqc2RT, tqc3RT]
#     # best = RTs.index(min(tqc3RT, tqc2RT, tqc1RT, tqc0RT))


#     print("Simulated Times: ", RTs)
#     # print("Accuracies: ", accuracies)
#     print("Simulated GCs: ", GCs)
#     print("Simulated Costs: ", costs) 

#     RTs2 = [RTs[0], RTs[1], RTs[2], RTs[3]]
#     GCs2 = [GCs[0], GCs[1], GCs[2], GCs[3]]
#     costs2 = [costs[0], costs[1], costs[2], costs[3]]

#     optLvls = [0, 1, 2, 3]
#     worst = RTs.index(max(RTs))
#     optLvls.pop(worst)
#     RTs.pop(worst)
#     GCs.pop(worst)
#     costs.pop(worst)
#     #all items are identical
#     if check(costs):
#         worst = RTs.index(max(RTs))
#     else:
#           worst = costs.index(max(costs))
#     optLvls.pop(worst)
#     RTs.pop(worst)
#     GCs.pop(worst)
#     costs.pop(worst)
#     if check(GCs):
#         worst = RTs.index(max(RTs))
#     else:
#         worst = GCs.index(max(GCs))
#     optLvls.pop(worst)
#     best = optLvls[0]
#     print("old best: ", best)
#     #Check if any RTs are too close to be distinguished
#     alternatives = []
#     for i in range(len(RTs2)):
#         if i != best:
#             # print(i)
#             # print((RTs2[i] - RTs2[best]) / RTs2[best] *100)
#             if (RTs2[i] - RTs2[best]) / RTs2[best] *100 <= 30 and GCs2[i] < GCs2[best] and costs2[i] < costs2[best]:
#                 best = i
#             elif (RTs2[i] - RTs2[best]) / RTs2[best] *100 <= 5 and GCs2[i] <= GCs2[best] and costs2[i] <= costs2[best]:
#                 alternatives.append(i)
#     # print("RTs: ", RTs)
#     print("Chosen Sim Optimization Level: ", best)
#     print("Alternative Optimization Lvls: ", alternatives)
#     # simTimes.append(time.time() - start)
#     simVal.append(best)
#     simAlts.append(alternatives)